In [2]:
import formulas
import utilities
import pandas as pd
import os
import re

def get_calculus_summary_display_string(input_series, **kwarg):
    """Returns a calculus summmary formula table in markdown format with 
    embedded html.  Input series is a pandas series with Indices State, 
    Subect code and category.  **Kwarg must be called with 
    parameter  = formulas_df where formulas_df contains fields State, 
    Subject code, Category, Formula_1, Formula_2.  Generates seperate tabs 
    to highlight items on formula sheet if there are any"""
    
    df = kwarg['formulas_df'].copy()
    df = df[(
        (df['State'] == str(input_series['State'])) &
        (df['Formula sub category 2'] == str(
            input_series['Formula sub category 2'])) &
        (df['Subject code'] == str(input_series['Subject code'])))]

    formula_sheet_list =kwarg.get('formula_sheet_list')
    
    output_string='#  \n<br>\n'
    calculus_df = formulas.calculus_summary_df(df)
    calculus_styler = formulas.calculus_summary_styler(calculus_df)

    output_string+=calculus_styler.to_html()
    
    return(output_string)
    



def create_formulas_content(website_creator_dir, docs_dir):

    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formula_sheet_list = formulas.get_formulas_on_formula_sheet(
        formulas_input_df)
    
    by_year_df =  (
        formulas.get_formulas_by_year_df(formulas_input_df))
    cumulative_df = (
        formulas.get_formulas_by_year_cumulative_df(
            formulas_input_df, sort_orders_df))
    
    formulas_df = pd.concat([by_year_df, cumulative_df])

    dirs_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Formula_1', 'Formula_2'])
                          

    calculus_summary_file_paths_df = (
        formulas.calculus_summary_file_paths_df(cumulative_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= calculus_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=get_calculus_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Differentiation', 'Integration'])
    
    display(calculus_summary_file_paths_df)


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(website_creator_dir, docs_dir)


Category,State,Formula sub category 1,Formula sub category 2,Subject code,Category
1,NSW,Formulas,By year cumulative,Year 12 Adv,Calculus summary
2,NSW,Formulas,By year cumulative,Year 12 Ext 1,Calculus summary


In [8]:
# import formulas
# import utilities
# import pandas as pd
# import os
# import re



# if __name__ == '__main__':
#     website_creator_dir = os.getcwd()
#     docs_dir = utilities.get_docs_path(website_creator_dir)

#     formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
#     order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
#     formulas_input_df = pd.read_csv(formula_file_path)
#     sort_orders_df = pd.read_csv(order_file_path)
#     formula_sheet_list = formulas.get_formulas_on_formula_sheet(
#         formulas_input_df)
    
#     by_year_df =  (
#         formulas.get_formulas_by_year_df(formulas_input_df))
#     cumulative_df = (
#         formulas.get_formulas_by_year_cumulative_df(
#             formulas_input_df, sort_orders_df))
    
#     formulas_df = pd.concat([by_year_df, cumulative_df])

#     dirs_df = (
#         formulas_df[['State', 'Formula sub category 1', 
#                              'Formula sub category 2',
#                              'Subject code']].drop_duplicates())
#     file_paths_df = (
#         formulas_df[['State', 'Formula sub category 1', 
#                              'Formula sub category 2',
#                              'Subject code', 'Category']].drop_duplicates())
    
#     utilities.create_sub_directories_from_df(base_dir = docs_dir, 
#                                              sub_paths_df = dirs_df)
    
#     front_matter_index_files = {'bookCollapseSection' : True}
#     utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
#                        front_matter=front_matter_index_files,
#                        sort_orders_df = sort_orders_df)
    
#     utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
#                            file_extension='.md', 
#                            fn=formulas.get_formula_display_string, 
#                            sort_orders_df = sort_orders_df,
#                            formulas_df = formulas_df, 
#                            formula_sheet_list = formula_sheet_list, 
#                            cols_to_highlight_if_in_formula_sheet = [
#                                'Formula_1', 'Formula_2']
                          

#     calculus_summary_file_paths_df = (
#         formulas.calculus_summary_file_paths_df(cumulative_df))

#     calculus_df = formulas.calculus_df_summary(cumulative_df)
#     calculus_styler = formulas.styler_calculus_summary(calculus_df, formula_sheet_list)
    

#     display(calculus_summary_file_paths_df)
#     display(calculus_styler)

